In [ ]:
import cobra
from cobra.flux_analysis import flux_variability_analysis
import matplotlib as plt
import pandas as pd
import csv


# Assignment 1: Metabolic Modelling

## Task 1

- **A. In the interactive session, we saw reaction fluxes in a linear pathway being equal to each other due to mass balance constraints. Do you observe the same thing now for the maximal reaction activities? Explain your observations. [5 pt]**

    In this case, the maximal reaction activities in a linear pathway are not equal to each other. If for example we look at the lower glycolysis pathway we get the following sequence of reaction activities:
    - GAPD: 26.1
    - PGK: 23.1
    - PGM: 20.0
    - ENO: 28.8

- **B. Some reactions have no maximal reaction activity data. Identify at least two different kinds of such reactions, and explain why gene expression-derived data would not be applicable for these kinds of reactions. [5 pt]**

    The first obvious reaction type are exchange reactions. If we specifically look at Ex_glc_D_e and EX_fru_e for example, they are empty and have no data. These specific reactions show the transport of that substance across the cell boundary. Gene expression data is not useful in this case for maximal activity as their rate is not only dependant on the number of expressed transport proteins. Also if there is no glucose or fructose in this case coming from an external environment the reaction rate is always zero.   

    The second reaction type would be Biomass, in this case we very clearly have BIOMASS_Ecoli_cor_w_GAM that has no data at all. The process it describes the one of cell growth, using precursor metabolites to simulate that. Since there is no gene that encodes such a process it is therefore not possible to get a maximal activity value from it only based on gene expression data. 

## Task 2

In [ ]:
import cobra
import csv
import pandas as pd
from cobra.flux_analysis import flux_variability_analysis

model = cobra.io.load_json_model('e_coli_core.json')

filename = 'e_coli_core_expression.csv'
estimated_maximal_activity = {}

with open(filename, mode='r', newline='') as file:
    reader = csv.reader(file)   
    for row in reader:
        
        try:
            num = float(row[1])
            estimated_maximal_activity[row[0]] = num
        except ValueError:
            continue

high_absolute_default_bound = 1000

for reaction in model.reactions:
    
    if (reaction.id=="ATPM"):
        print("ATPM reaction unchanged")
        continue
    
    if (reaction.id=="EX_glc__D_e"):
        print("EX_glc__D_e bounds changed to high absolute default bound")
        reaction.lower_bound = -high_absolute_default_bound
        reaction.upper_bound = high_absolute_default_bound
        continue
    
    try:
        value = estimated_maximal_activity[reaction.id]
    except KeyError:
        continue
    
    if (reaction.reversibility == True):
        reaction.lower_bound = -value
        reaction.upper_bound = value
        
    elif (reaction.reversibility == False):
        reaction.lower_bound = 0.0
        reaction.upper_bound = value

In [ ]:
for reaction in model.reactions:
    print(f'{reaction.id}: UB:{reaction.upper_bound}, LB:{reaction.lower_bound}')

## Task 3

## Task 4

In [ ]:
# 4.a
# Perform FBA optimization
fba_results = model.optimize()

# Report the maximal biomass production rate
print(f"Maximal biomass production rate: {fba_results.objective_value:.4f}")

In [ ]:
# 4.b: determine reaction whose flux reaches its maximum
reactions_at_max_flux = []
for reaction in model.reactions:
    flux = fba_results.fluxes[reaction.id]
    if flux == reaction.upper_bound:
        reactions_at_max_flux.append(reaction)

print(f"Number of reactions whose flux reaches its maximum: {len(reactions_at_max_flux)}")
print("Reactions whose flux reaches its maximum:")
for reaction in reactions_at_max_flux:
    print(f"- {reaction.id}")   

In [ ]:
#  4.c
print("Reactions with given maximal reaction activity and zero flux:")
list_4c = []
for reaction_id in estimated_maximal_activity.keys():
    reaction = model.reactions.get_by_id(reaction_id)
    if fba_results.fluxes[reaction.id] == 0:
        list_4c.append(reaction.id)
print(list_4c)


### 4.c

The reaction we are going to analyze is 'PFL'. When we look at the reaction in the online demo we see that it is a reaction which only connects two point and runs in parallel with another reaction, 'PDH'. This also immediately explains why 'PFL' has no flux. All flux that is needed between the two nodes is provided by the other reaction. This make 'PFL' redundant. 

![image.png](Q4c.png)